In [59]:
import tensorflow as tf
import tensorflow_datasets as tfds
import math
import numpy as np
import matplotlib.pyplot as plt
import logging
from __future__ import absolute_import, division, print_function, unicode_literals
from tkinter import *
from PIL import Image, ImageDraw, ImageOps
import io
import tkinter as tk

In [60]:

logger = tf.get_logger()
logger.setLevel(logging.ERROR)

dataset, metadata = tfds.load('mnist', as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

class_names = [
    'Cero', 'Uno', 'Dos', 'Tres', 'Cuatro', 'Cinco', 'Seis',
    'Siete', 'Ocho', 'Nueve'
]

num_train_examples = metadata.splits['train'].num_examples
num_test_examples = metadata.splits['test'].num_examples

In [61]:
#Normalizar: Numeros de 0 a 255, que sean de 0 a 1
def normalize(images, labels):
    images = tf.cast(images, tf.float32)
    images /= 255
    return images, labels

train_dataset = train_dataset.map(normalize)
test_dataset = test_dataset.map(normalize)

In [62]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28, 1)),
    
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),  # Regularización
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(10, activation='softmax')
])

In [63]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [64]:
#Indicar las funciones a utilizar
model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [65]:
#Aprendizaje por lotes de 32 cada lote
BATCHSIZE = 32
train_dataset = train_dataset.repeat().shuffle(num_train_examples).batch(BATCHSIZE)
test_dataset = test_dataset.batch(BATCHSIZE)

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    patience=3,  # Detener si no mejora en 3 épocas
    restore_best_weights=True
)



In [66]:

history = model.fit(
    train_dataset,
    epochs=20,  # Más épocas
    steps_per_epoch=math.ceil(metadata.splits['train'].num_examples / BATCHSIZE),
    validation_data=test_dataset,
    validation_steps=math.ceil(metadata.splits['test'].num_examples / BATCHSIZE),
    callbacks=[early_stopping]
)

test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"\n Precisión en el conjunto de prueba: {test_accuracy * 100:.2f}%")

print("Resultado en las pruebas: ", test_accuracy)
for test_images, test_labels in test_dataset.take(1):
	test_images = test_images.numpy()
	test_labels = test_labels.numpy()
	predictions = model.predict(test_images)


Epoch 1/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8485 - loss: 0.4944 - val_accuracy: 0.9555 - val_loss: 0.1429
Epoch 2/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9515 - loss: 0.1571 - val_accuracy: 0.9640 - val_loss: 0.1119
Epoch 3/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9645 - loss: 0.1199 - val_accuracy: 0.9717 - val_loss: 0.0911
Epoch 4/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9697 - loss: 0.0993 - val_accuracy: 0.9751 - val_loss: 0.0798
Epoch 5/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9721 - loss: 0.0858 - val_accuracy: 0.9758 - val_loss: 0.0878
Epoch 6/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9753 - loss: 0.0783 - val_accuracy: 0.9778 - val_loss: 0.0784
Epoch 7/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9773 - loss: 0.0748 - val_accuracy: 0.9765 - val_loss: 0.0825
Epoch 8/20
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9801 - loss: 0.0662 - 

In [71]:
class AplicacionDibujo:
    def __init__(self, modelo):
        self.ventana = tk.Tk()
        self.ventana.title("Dibuja un número (0-9)")

        self.ancho_canvas = 200
        self.alto_canvas = 200
        self.canvas = tk.Canvas(self.ventana, width=self.ancho_canvas, height=self.alto_canvas, bg='white')
        self.canvas.pack()

        self.boton_predecir = tk.Button(self.ventana, text="Predecir", command=self.predecir_numero)
        self.boton_predecir.pack()

        self.boton_limpiar = tk.Button(self.ventana, text="Limpiar", command=self.limpiar_canvas)
        self.boton_limpiar.pack()

        self.etiqueta_resultado = tk.Label(self.ventana, text="", font=("Helvetica", 16))
        self.etiqueta_resultado.pack()

        self.modelo = modelo

        self.canvas.bind("<B1-Motion>", self.dibujar)
        self.imagen = Image.new("L", (self.ancho_canvas, self.alto_canvas), 255)
        self.dibujo_imagen = ImageDraw.Draw(self.imagen)

    def dibujar(self, evento):
        x, y = evento.x, evento.y
        radio = 8  # grosor del trazo
        self.canvas.create_oval(x - radio, y - radio, x + radio, y + radio, fill='black', outline='black')
        self.dibujo_imagen.ellipse([x - radio, y - radio, x + radio, y + radio], fill='black')

    def limpiar_canvas(self):
        self.canvas.delete("all")
        self.dibujo_imagen.rectangle([0, 0, self.ancho_canvas, self.alto_canvas], fill='white')
        self.etiqueta_resultado.config(text="")

    def preprocesar_imagen(self):
        # Convertir a 28x28, invertir colores
        imagen_redimensionada = self.imagen.resize((28, 28))
        imagen_invertida = ImageOps.invert(imagen_redimensionada)

        # Mejorar contraste
        imagen_invertida = ImageOps.autocontrast(imagen_invertida)

        # Convertir a array numpy
        imagen_array = np.array(imagen_invertida)

        # Eliminar ruido
        imagen_array = np.where(imagen_array > 50, imagen_array, 0)

        # Centrar la imagen
        coordenadas = np.column_stack(np.where(imagen_array > 0))
        if coordenadas.size:
            y0, x0 = coordenadas.min(axis=0)
            y1, x1 = coordenadas.max(axis=0)
            imagen_array = imagen_array[y0:y1 + 1, x0:x1 + 1]

        # Redimensionar a 20x20 y centrar en 28x28
        imagen_pil = Image.fromarray(imagen_array).resize((20, 20), Image.LANCZOS)
        imagen_centrada = Image.new('L', (28, 28), 0)
        imagen_centrada.paste(imagen_pil, ((28 - 20) // 2, (28 - 20) // 2))

        # Normalizar
        imagen_array = np.array(imagen_centrada).astype(np.float32) / 255.0
        imagen_array = imagen_array.reshape(1, 28, 28, 1)

        return imagen_array

    def predecir_numero(self):
        imagen_procesada = self.preprocesar_imagen()
        prediccion = self.modelo.predict(imagen_procesada)

        etiqueta = np.argmax(prediccion)
        confianza = prediccion[0][etiqueta]
        self.etiqueta_resultado.config(text=f"Predicción: {class_names[etiqueta]} ({confianza * 100:.2f}%)")

    def ejecutar(self):
        self.ventana.mainloop()


# Lanzar la aplicación
app = AplicacionDibujo(model)
app.ejecutar()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


In [74]:
model.save("modelo_mnist.h5")


#esto lo vamos a usar en anaconda prompt
(base) C:\Users\carlo>cd Red_Neuronal_01

(base) C:\Users\carlo\Red_neuronal_01>uvicorn main:app --reload